In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%set_env CUDA_LAUNCH_BLOCKING=1

env: CUDA_LAUNCH_BLOCKING=1


In [3]:
import sys
import collections
import cv2
import os
import math
import random
import pickle
import copy
import numpy as np
import pandas as pd
sys.path.append("..")
sys.path.append(".")


In [4]:
sys.path.remove('/Users/amrutamuthal/Documents/VSWorkspaces/LayoutFormer++/LayoutGeneration/LayoutFormer++/src')

In [5]:
import torch
import torch.optim as optim
from torch_geometric.data import Data
from torch_geometric.utils import dense_to_sparse
from torch_geometric.data import DataLoader
import torch.utils.data as data_utils
from torch.utils.tensorboard import SummaryWriter

In [6]:
os.chdir('/Users/amrutamuthal/Documents/VSWorkspaces/MeronymNet-PyTorch')

In [9]:
from losses import BoxVAE_losses as loss
from evaluation import metrics
from utils import plot_utils
from utils import data_utils as data_loading
from components.DenseAutoencoder import DenseAutoencoder
from components.DenseAutoencoder import Decoder
from components.TwoStageAutoEncoder import TwoStageAutoEncoder

In [8]:
# sys.path#.remove('/Users/amrutamuthal/Documents/VSWorkspaces/LayoutFormer++/LayoutGeneration/LayoutFormer++/src')
import matplotlib.pyplot as plt

In [9]:
# from mask_generation import masked_sketch

In [12]:
batch_size = 128
latent_dims = 64
num_nodes = 16
bbx_size = 4
num_classes = 7
label_shape = 1
nb_epochs = 1000
klw = loss.frange_cycle_linear(nb_epochs)
learning_rate = 0.00008
hidden1 = 32
hidden2 = 16
hidden3 = 128
dense_hidden1=8
dense_hidden2=4
adaptive_margin = True
fine_tune_box = False
output_log = False
area_encoding = False
run_prefix = "two_stage_small_obj_conditioning_fft_gcn_reccurent_refine_coarse_bbx_loss_hw_variational"
variational=True
coupling = True
obj_bbx_conditioning = True
use_fft_on_bbx = True
use_gcn_in_decoder = True

In [36]:
batch_train_loader, batch_val_loader = data_loading.load_data(obj_data_postfix = '_obj_boundary_sqr',
                                                              part_data_post_fix = '_scaled_sqr',
                                                              file_postfix = '_combined',
                                                              seed=345,
                                                              batch_size=batch_size)

/opt/anaconda3/lib/python3.12/site-packages/torch_geometric/deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


In [37]:
import gc
gc.collect()

0

In [38]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

In [39]:
del vae

In [15]:
run_prefix

'two_stage_small_obj_conditioning_fft_gcn_reccurent_refine_coarse_bbx_loss_hw_variational'

In [40]:
reconstruction_loss_arr = []
kl_loss_obj_arr = []
kl_loss_part_arr = []
bbox_loss_arr = []
refined_bbox_loss_arr = []
adj_loss_arr = []
node_loss_arr = []

reconstruction_loss_val_arr = []
kl_loss_val_arr = []
bbox_loss_val_arr = []
refined_bbox_loss_val_arr = []
adj_loss_val_arr = []
node_loss_val_arr = []

bbox_loss_threshold = 1.0
vae = TwoStageAutoEncoder(latent_dims,
                          num_nodes,
                          bbx_size,
                          num_classes,
                          label_shape,
                          hidden1,
                          hidden2,
                          hidden3,
                          dense_hidden1,
                          dense_hidden2,
                          adaptive_margin,
                          output_log,
                          area_encoding,
                          coupling,
                          obj_bbx_conditioning,
                          use_fft_on_bbx,
                          use_gcn_in_decoder
                        )
vae.to(device)
optimizer = optim.Adam(vae.parameters(), lr=learning_rate)
scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[100,300], gamma=0.75)
model_path = ('/Users/amrutamuthal/Documents/training_runs/model/'+run_prefix+'/Obj-Box-'
                        +str(learning_rate)
                        +'-batch-'+str(batch_size)
                        +'-h1-'+str(hidden1)
                        +'-h2-'+str(hidden2)
                        +'-h3-'+str(hidden3)+'-test')
summary_path = ('/Users/amrutamuthal/Documents/training_runs/runs/'+run_prefix+'/Obj-Box-'
                        +str(learning_rate)
                        +'-batch-'+str(batch_size)
                        +'-h1-'+str(hidden1)
                        +'-h2-'+str(hidden2)
                        +'-h3-'+str(hidden3)+'-test')
if not os.path.exists(model_path):
    os.makedirs(model_path)
writer = SummaryWriter(summary_path)
icoef = 0

for epoch in range(nb_epochs):
    
    batch_loss = torch.tensor([0.0])
    batch_kl_loss_part = torch.tensor([0.0])
    batch_kl_loss_obj = torch.tensor([0.0])
    batch_bbox_loss = torch.tensor([0.0])
    batch_refined_bbox_loss = torch.tensor([0.0])
    batch_obj_bbox_loss = torch.tensor([0.0])
    batch_node_loss = torch.tensor([0.0])
    IOU_weight_delta = torch.tensor([(1+epoch)/nb_epochs])
    images = []
    
    vae.train()
    i=0
    for train_data in batch_train_loader:
        
        node_data_true = train_data.x
        label_true = node_data_true[:,:1]
        y_true = train_data.y
        class_true = y_true[:, :num_classes]
        X_obj_true = y_true[:, num_classes:]
        X_obj_true_transformed = torch.ones((X_obj_true.shape[0], 2))*0.5
        # X_obj_true_transformed = X_obj_true[:,2:]-X_obj_true[:,:2]
        node_data_transformed = torch.cat(
            [node_data_true[:, :-2],
             node_data_true[:, -2:] -  node_data_true[:, -4:-2]], axis=-1)

        adj_true = train_data.edge_index
        batch = train_data.batch
        
        class_true  = torch.flatten(class_true)
        

        for param in vae.parameters():
            param.grad=None
        
        output = vae(
            adj_true,
            node_data_transformed,
            X_obj_true_transformed,
            label_true,
            class_true, variational, coupling, refine_iter=2
            # training=(epoch>100)
            )
        
        node_data_pred = output[0]
        node_data_pred_new = torch.cat(
            [node_data_pred[:, :, :-2],
             node_data_pred[:, :, -2:] + node_data_pred[:, :, -4:-2]], axis=-1)
        X_obj_pred = output[1]
        X_obj_pred = torch.cat(((torch.tensor([1.0])-X_obj_pred)*torch.tensor([0.5]),
                                (torch.tensor([1.0])+X_obj_pred)*torch.tensor([0.5])),
                              -1)
        label_pred = output[2]
        z_mean_part = output[3]
        z_logvar_part = output[4]
        margin = output[5]
        z_mean_obj = output[6]
        z_logvar_obj = output[7]

        node_data_pred_refined = output[8]
        node_data_pred_refined_new = torch.cat(
            [node_data_pred_refined[:, :, :-2],
             node_data_pred_refined[:, :, -2:] + node_data_pred_refined[:, :, -4:-2]], axis=-1)

        obj_bbox_loss = loss.obj_bbox_loss(pred_box=X_obj_pred, true_box=X_obj_true, weight=IOU_weight_delta, has_mse=False)
        kl_loss_obj = loss.kl_loss(z_mean_obj, z_logvar_obj)
        kl_loss_part = loss.kl_loss(z_mean_part, z_logvar_part)
        bbox_loss = loss.weighted_bbox_loss(pred_box=node_data_pred_new, true_box=node_data_true[:,1:], weight=IOU_weight_delta, margin=margin)
        bbox_loss_coarse = loss.coarse_bbx_loss(pred_box=node_data_pred_new, true_box=node_data_true[:,1:])
        node_loss = loss.node_loss(label_pred,label_true)
        
        # reconstruction_loss = (node_loss)*num_nodes

        if use_gcn_in_decoder:
            refined_bbox_loss = loss.weighted_bbox_loss(
                pred_box=node_data_pred_refined_new,
                true_box=node_data_true[:, 1:],
                weight=IOU_weight_delta,
                margin=margin,
            )
            reconstruction_loss = (refined_bbox_loss + node_loss) * num_nodes # * epoch / nb_epochs
        
        else:
            refined_bbox_loss = torch.tensor([0.0])
        
        kl_weight = klw[icoef]
        if variational and (kl_weight>0):
            reconstruction_loss += kl_loss_part*kl_weight   
        
        if epoch >50:
            reconstruction_loss += bbox_loss_coarse

        reconstruction_loss.backward()
        
        optimizer.step()
        
        i+=1
      
        batch_loss += reconstruction_loss
        batch_kl_loss_part += kl_loss_part
        batch_kl_loss_obj += kl_loss_obj
        batch_bbox_loss += bbox_loss
        batch_refined_bbox_loss += refined_bbox_loss
        batch_obj_bbox_loss += obj_bbox_loss
        batch_node_loss += node_loss
    
        if i%200==0:
            print(i)
            global_step = epoch*len(batch_train_loader)+i
            
            writer.add_scalar("Loss/train/reconstruction_loss", batch_loss.item()/(i+1), global_step)
            writer.add_scalar("Loss/train/kl_loss_part", batch_kl_loss_part.item()/(i+1), global_step)
            writer.add_scalar("Loss/train/kl_loss_obj", batch_kl_loss_obj.item()/(i+1), global_step)
            writer.add_scalar("Loss/train/bbox_loss", batch_bbox_loss.item()/(i+1), global_step)
            writer.add_scalar("Loss/train/refined_bbox_loss", batch_refined_bbox_loss.item()/(i+1), global_step)
            writer.add_scalar("Loss/train/obj_bbox_loss", batch_obj_bbox_loss.item()/(i+1), global_step)
            writer.add_scalar("Loss/train/node_loss", batch_node_loss.item()/(i+1), global_step)
            
            
#     scheduler.step()
    global_step = epoch*len(batch_train_loader)+i
    image_shape = [num_nodes, bbx_size]

    image = plot_utils.plot_bbx(np.reshape((node_data_true[:num_nodes,1:5]*label_true[:num_nodes]).detach().to("cpu").numpy(),
                                image_shape)).astype(float)/255
    writer.add_image('train/images/input', image, global_step, dataformats='HWC')
    image = plot_utils.plot_bbx((node_data_pred_new[0]*label_true[:num_nodes]).detach().to("cpu").numpy()).astype(float)/255
    writer.add_image('train/images/generated', image, global_step, dataformats='HWC')
    image = plot_utils.plot_bbx((node_data_pred_refined_new[0]*label_true[:num_nodes]).detach().to("cpu").numpy()).astype(float)/255
    writer.add_image('train/images/refined', image, global_step, dataformats='HWC')
    
    reconstruction_loss_arr.append(batch_loss.detach().item()/(i+1))
    kl_loss_obj_arr.append(batch_kl_loss_obj.detach().item()/(i+1))
    kl_loss_part_arr.append(batch_kl_loss_part.detach().item()/(i+1))
    bbox_loss_arr.append(batch_bbox_loss.detach().item()/(i+1))
    refined_bbox_loss_arr.append(batch_refined_bbox_loss.detach().item()/(i+1))
    node_loss_arr.append(batch_node_loss.detach().item()/(i+1))
    
    print('[%d, %5d] loss: %.3f' %
              (epoch + 1, i + 1, batch_loss/(i+1) ))
    
    
    batch_loss = torch.tensor([0.0])
    batch_kl_loss_part = torch.tensor([0.0])
    batch_kl_loss_obj = torch.tensor([0.0])
    batch_bbox_loss = torch.tensor([0.0])
    batch_refined_bbox_loss = torch.tensor([0.0])
    batch_obj_bbox_loss = torch.tensor([0.0])
    batch_node_loss = torch.tensor([0.0])
    images = []
    vae.eval()
    for i, val_data in enumerate(batch_val_loader, 0):
        
        node_data_true = val_data.x
        label_true = node_data_true[:,:1]
        y_true = val_data.y
        class_true = torch.flatten(y_true[:, :num_classes])
        X_obj_true = y_true[:, num_classes:]
        X_obj_true_transformed = torch.ones((X_obj_true.shape[0], 2))*0.5
        # X_obj_true_transformed = X_obj_true[:,2:]-X_obj_true[:,:2]
        node_data_transformed = torch.cat(
            [node_data_true[:, :-2],
             node_data_true[:, -2:] -  node_data_true[:, -4:-2]], axis=-1)
        adj_true = val_data.edge_index
        batch = val_data.batch
        
        class_true  = torch.flatten(class_true)
        
        output = vae(adj_true, node_data_transformed, X_obj_true_transformed, label_true , class_true, variational, coupling)
        
        node_data_pred = output[0]
        node_data_pred_new = node_data_pred
        node_data_pred_new = torch.cat(
            [node_data_pred[:, :, :-2],
             node_data_pred[:, :, -2:] + node_data_pred[:, :, -4:-2]], axis=-1)

        X_obj_pred = output[1]
        X_obj_pred = torch.cat(((torch.tensor([1.0])-X_obj_pred)*torch.tensor([0.5]),
                                (torch.tensor([1.0])+X_obj_pred)*torch.tensor([0.5])),
                              -1)
        label_pred = output[2]
        z_mean_part = output[3]
        z_logvar_part = output[4]
        margin = output[5]
        z_mean_obj = output[6]
        z_logvar_obj = output[7]

        node_data_pred_refined = output[8]
        node_data_pred_refined_new = node_data_pred_refined
        node_data_pred_refined_new = torch.cat(
            [node_data_pred_refined_new[:, :, :-2],
             node_data_pred_refined_new[:, :, -2:] + node_data_pred_refined_new[:, :, -4:-2]], axis=-1)
        
        
        obj_bbox_loss = loss.obj_bbox_loss(pred_box=X_obj_pred, true_box=X_obj_true, weight=IOU_weight_delta, has_mse=False)
        kl_loss_obj = loss.kl_loss(z_mean_obj, z_logvar_obj)
        kl_loss_part = loss.kl_loss(z_mean_part, z_logvar_part)
        bbox_loss = loss.weighted_bbox_loss(pred_box=node_data_pred_new, true_box=node_data_true[:,1:], weight=IOU_weight_delta, margin=margin)
        bbox_loss_coarse = loss.coarse_bbx_loss(pred_box=node_data_pred_new, true_box=node_data_true[:,1:])
        refined_bbox_loss = loss.weighted_bbox_loss(pred_box=node_data_pred_refined_new, true_box=node_data_true[:,1:], weight=IOU_weight_delta, margin=margin)
        node_loss = loss.node_loss(label_pred,label_true)
        
        if kl_weight>0:
            reconstruction_loss = kl_loss_part*kl_weight + (bbox_loss + node_loss)*num_nodes
        else:
            reconstruction_loss = (refined_bbox_loss + node_loss)*num_nodes

        if epoch > 50:
            reconstruction_loss +=  refined_bbox_loss
            
        batch_loss += reconstruction_loss
        batch_kl_loss_part += kl_loss_part
        batch_kl_loss_obj += kl_loss_obj
        batch_bbox_loss += bbox_loss
        batch_refined_bbox_loss += refined_bbox_loss
        batch_obj_bbox_loss += obj_bbox_loss
        batch_node_loss += node_loss
    
    image = plot_utils.plot_bbx(np.reshape((node_data_true[:num_nodes,1:5]*label_true[:num_nodes]).detach().to("cpu").numpy(),
                                image_shape)).astype(float)/255
    writer.add_image('val/images/input', image, global_step, dataformats='HWC')
    image = plot_utils.plot_bbx((node_data_pred_new[0]*label_true[:num_nodes]).detach().to("cpu").numpy()).astype(float)/255
    writer.add_image('val/images/generated', image, global_step, dataformats='HWC')
    image = plot_utils.plot_bbx((node_data_pred_refined_new[0]*label_true[:num_nodes]).detach().to("cpu").numpy()).astype(float)/255
    writer.add_image('val/images/refined', image, global_step, dataformats='HWC')

    reconstruction_loss_val_arr.append(batch_loss.detach().item()/(i+1))
    bbox_loss_val_arr.append(batch_bbox_loss.detach().item()/(i+1))
    node_loss_val_arr.append(batch_node_loss.detach().item()/(i+1))
    
    writer.add_scalar("Loss/val/reconstruction_loss", batch_loss.detach()/(i+1), global_step)
    writer.add_scalar("Loss/train/kl_loss_part", batch_kl_loss_part.item()/(i+1), global_step)
    writer.add_scalar("Loss/train/kl_loss_obj", batch_kl_loss_obj.item()/(i+1), global_step)
    writer.add_scalar("Loss/val/bbox_loss", batch_bbox_loss.item()/(i+1), global_step)
    writer.add_scalar("Loss/val/refined_bbox_loss", batch_refined_bbox_loss.item()/(i+1), global_step)
    writer.add_scalar("Loss/val/obj_bbox_loss", batch_obj_bbox_loss.item()/(i+1), global_step)
    writer.add_scalar("Loss/val/node_loss", batch_node_loss.item()/(i+1), global_step)
       
    if epoch%50 == 0:
        torch.save(vae.state_dict(), model_path + '/model_weights.pth')
        
    if ((kl_loss_part_arr[-1]>0.5) and 
        (abs(bbox_loss_arr[-1] - bbox_loss_val_arr[-1]) < 0.1) and 
        (bbox_loss_arr[-1]<bbox_loss_threshold) and (epoch>300)):
        
        icoef = icoef + 1
        bbox_loss_threshold*=0.9

torch.save(vae.state_dict(),model_path + '/model_weights.pth')

for i in range(min(100,int(len(node_data_true)/num_nodes))):    
    image = plot_utils.plot_bbx(np.reshape((node_data_true[num_nodes*i:num_nodes*(i+1),1:5]).detach().to("cpu").numpy(),
                                    image_shape)).astype(float)/255
    writer.add_image('result/images/'+str(i)+'-input', image, global_step, dataformats='HWC')
    image = plot_utils.plot_bbx((node_data_pred[i]*label_true[num_nodes*i:num_nodes*(i+1)]).detach().to("cpu").numpy()).astype(float)/255
    writer.add_image('result/images/'+str(i)+'-generated', image, global_step, dataformats='HWC')
    
writer.flush()
writer.close()
print('Finished Training')

200
[1,   399] loss: 502.045
200
[2,   399] loss: 225.156
200
[3,   399] loss: 197.368
200
[4,   399] loss: 186.912
200
[5,   399] loss: 180.049
200
[6,   399] loss: 174.321
200
[7,   399] loss: 168.896
200
[8,   399] loss: 163.566
200
[9,   399] loss: 158.359
200
[10,   399] loss: 153.292
200
[11,   399] loss: 148.348
200
[12,   399] loss: 143.610
200
[13,   399] loss: 139.079
200
[14,   399] loss: 134.758
200
[15,   399] loss: 130.647
200
[16,   399] loss: 126.740
200
[17,   399] loss: 123.021
200
[18,   399] loss: 119.484
200
[19,   399] loss: 116.125
200
[20,   399] loss: 112.901
200
[21,   399] loss: 109.780
200
[22,   399] loss: 106.419
200
[23,   399] loss: 102.754
200
[24,   399] loss: 99.359
200
[25,   399] loss: 96.106
200
[26,   399] loss: 93.012
200
[27,   399] loss: 90.079
200
[28,   399] loss: 87.332
200
[29,   399] loss: 84.597
200
[30,   399] loss: 82.090
200
[31,   399] loss: 79.769
200
[32,   399] loss: 77.545
200
[33,   399] loss: 75.548
200
[34,   399] loss: 73.691


In [41]:
 sum(p.numel() for p in vae.parameters() if p.requires_grad)

173261

In [19]:
#testing loop
model_path = ('/Users/amrutamuthal/Documents/training_runs/model/'+run_prefix+'/Obj-Box-'
                        +str(learning_rate)
                        +'-batch-'+str(batch_size)
                        +'-h1-'+str(hidden1)
                        +'-h2-'+str(hidden2)
                        +'-h3-'+str(hidden3)+'-test')
summary_path = ('/Users/amrutamuthal/Documents/training_runs/run/'+run_prefix+'/Obj-Box-'
                        +str(learning_rate)
                        +'-batch-'+str(batch_size)
                        +'-h1-'+str(hidden1)
                        +'-h2-'+str(hidden2)
                        +'-h3-'+str(hidden3)+'-test')


write_tensorboard = False
if write_tensorboard:
    writer = SummaryWriter(summary_path)

vae = TwoStageAutoEncoder(latent_dims,
                          num_nodes,
                          bbx_size,
                          num_classes,
                          label_shape,
                          hidden1,
                          hidden2,
                          hidden3,
                          dense_hidden1,
                          dense_hidden2,
                          adaptive_margin,
                          output_log,
                          area_encoding,
                          coupling,
                          obj_bbx_conditioning,
                          use_fft_on_bbx,
                          use_gcn_in_decoder,
                        )

vae.load_state_dict(torch.load(model_path+ '/model_weights.pth'))

# decoder = vae.decoder
image_shape = [num_nodes, bbx_size]
global_step = 250000
class_dict = {0:'cow', 1:'sheep', 2:'bird', 3:'person', 4:'cat', 5:'dog', 6:'horse'}
res_dfs = []
for i, val_data in enumerate(batch_val_loader, 0):
    
    val_data
    node_data_true = val_data.x
    label_true = node_data_true[:,:1]
    y_true = val_data.y
    class_true = y_true[:, :num_classes]
    X_obj_true = y_true[:, num_classes:]
    X_obj_true_transformed = X_obj_true[:,2:]-X_obj_true[:,:2]
    node_data_transformed = torch.cat(
            [node_data_true[:, :-2],
             node_data_true[:, -2:] -  node_data_true[:, -4:-2]], axis=-1)
    adj_true = val_data.edge_index
    class_true  = torch.flatten(class_true)
    
    output = vae(adj_true, node_data_transformed, X_obj_true_transformed, label_true , class_true, variational, coupling, refine_iter=2)
    node_data_pred_test = output[0]
    node_data_pred_test_refined = output[8]
    node_data_pred_test_refined_new = torch.clip(torch.cat(
            [node_data_pred_test_refined[:, :, :-2],
             node_data_pred_test_refined[:, :, -2:] + node_data_pred_test_refined[:, :, -4:-2]], axis=-1), min=0.0, max=1.0)
    X_obj_pred_test = output[1]
#     node_data_pred_test, X_obj_pred_test, label_pred, z_mean, z_logvar, margin = output
    X_obj_pred_test = torch.cat(((torch.tensor([1.0])-X_obj_pred_test)*torch.tensor([0.5]),
                                (torch.tensor([1.0])+X_obj_pred_test)*torch.tensor([0.5])),
                              -1)
    res_dfs.append(metrics.get_metrics(node_data_true, X_obj_true, node_data_pred_test_refined_new,
                               X_obj_pred_test,
                               label_true,
                               class_true,
                               num_nodes,
                               num_classes))
    
    if write_tensorboard:
        
        for j in range(int(len(node_data_true)/num_nodes)):
            
            image = plot_bbx(node_data_true[j].detach().to("mps").numpy().astype(float))/255
            pred_image = plot_bbx(node_data_pred_test[j].detach().to("mps").numpy()).astype(float)/255

            writer.add_image('test_result/images/'+str(j)+'-input/', image, global_step, dataformats='HWC')  
            writer.add_image('test_result/images/'+str(j)+'-generated/', pred_image, global_step, dataformats='HWC')
            

result = pd.concat(res_dfs)
result['obj_class'] = np.where(result['obj_class'].isna(), 0, result['obj_class'])
result['obj_class'] = result['obj_class'].astype('int')
result['obj_class'].replace(class_dict, inplace=True)
result.where(result['part_labels']!=0, np.NaN, inplace=True)
result['part_labels'] = np.where(result['part_labels'].isna(), 0, result['part_labels'])
result['part_labels'] = result['part_labels'].astype('int')
result['id'] = np.repeat(np.array(list(range(int(len(result)/num_nodes)))), 16)

if write_tensorboard:
    writer.flush()
    writer.close()

bird_labels = {'head':1 , 'torso':2, 'neck':3, 'lwing':4, 'rwing':5, 'lleg':6, 'lfoot':7, 'rleg':8, 'rfoot':9, 'tail':10}
cat_labels = {'head':1, 'torso':2, 'neck':3, 'lfleg':4, 'lfpa':5, 'rfleg':6, 'rfpa':7, 'lbleg':8, 'lbpa':9, 'rbleg':10, 'rbpa':11, 'tail':12}
cow_labels = {'head':1,'lhorn':2, 'rhorn':3, 'torso':4, 'neck':5, 'lfuleg':6, 'lflleg':7, 'rfuleg':8, 'rflleg':9, 'lbuleg':10, 'lblleg':11, 'rbuleg':12, 'rblleg':13, 'tail':14}
dog_labels = {'head':1,'torso':2, 'neck':3, 'lfleg':4, 'lfpa':5, 'rfleg':6, 'rfpa':7, 'lbleg':8, 'lbpa':9, 'rbleg':10, 'rbpa':11, 'tail':12, 'muzzle':13}
horse_labels = {'head':1,'lfho':2, 'rfho':3, 'torso':4, 'neck':5, 'lfuleg':6, 'lflleg':7, 'rfuleg':8, 'rflleg':9, 'lbuleg':10, 'lblleg':11, 'rbuleg':12, 'rblleg':13, 'tail':14, 'lbho':15, 'rbho':16}
person_labels = {'head':1, 'torso':2, 'neck': 3, 'llarm': 4, 'luarm': 5, 'lhand': 6, 'rlarm':7, 'ruarm':8, 'rhand': 9, 'llleg': 10, 'luleg':11, 'lfoot':12, 'rlleg':13, 'ruleg':14, 'rfoot':15}
sheep_labels = cow_labels
part_labels_combined_parts = {'bird': bird_labels, 'cat': cat_labels, 'cow': cow_labels, 'dog': dog_labels, 'sheep': sheep_labels, 'horse':horse_labels,'person':person_labels}

for k, v in part_labels_combined_parts.items():
    new_map = {}
    for pk, pv in v.items():
        new_map[pv]=pk
    part_labels_combined_parts[k] = new_map
    
for k, v in part_labels_combined_parts.items():
    result.loc[result['obj_class']==k, ['part_labels']] = result.loc[result['obj_class']==k,['part_labels']].replace(v).copy()

# rel_metrics.csv')sult.to_csv(model_path+'/raw_metrics.csv')
# res_obj_level = result.groupby(['obj_class', 'id']).mean().reset_index()
# res_obj_level.groupby(['obj_class']).mean().reset_index()[['obj_class', 'IOU', 'MSE']].to_csv(model_path+'/obj_level_metrics.csv')
# result.groupby(['obj_class', 'part_labels']).mean().reset_index()[['obj_class', 'part_labels',  'IOU', 'MSE']].to_csv(model_path+'/part_leve

/var/folders/ww/3nytfk1917b8nz7dchtckxnm0000gn/T/ipykernel_21700/3865254239.py:39: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  vae.load_state_dict(torch.load(model_path+ '

In [20]:
result[['obj_class', 'part_labels', 'IOU', 'MSE', 'id']].groupby(['obj_class', 'part_labels']).mean().reset_index()

,obj_class,part_labels,IOU,MSE,id
0,bird,head,0.385774,0.002879,3421.174733
1,bird,lfoot,0.098099,0.002726,3400.752688
2,bird,lleg,0.172394,0.002067,3446.555746
3,bird,lwing,0.241709,0.005186,3417.275556
4,bird,neck,0.233415,0.002188,3377.172485
...,...,...,...,...,...
81,sheep,lhorn,0.226874,0.002429,3340.952663
82,sheep,neck,0.428710,0.001199,3247.891593
83,sheep,rhorn,0.114240,0.003152,3474.713415
84,sheep,tail,0.166202,0.002189,3446.645283


In [21]:
res_obj_level = result[['obj_class', 'IOU', 'MSE', 'id']].groupby(['obj_class', 'id']).mean().reset_index()
res_obj_level.groupby(['obj_class']).mean().reset_index()[['obj_class', 'IOU', 'MSE']]


,obj_class,IOU,MSE
0,bird,0.357914,0.002372
1,cat,0.416449,0.007059
2,cow,0.382756,0.002814
3,dog,0.315267,0.003686
4,horse,0.279373,0.004577
5,person,0.376353,0.003188
6,sheep,0.490752,0.002206


In [13]:
def inference(X_obj, part_decoder, obj_class, nodes, batch_size, latent_dims, E, refine_iter):
    
    nodes = torch.reshape(nodes,(batch_size,part_decoder.num_nodes))
    obj_class = torch.reshape(obj_class, 
                              (batch_size, num_classes))
    
    # obj sampling
    z_latent_obj = X_obj
    
    print(z_latent_obj.shape, obj_class.shape)
    conditioned_obj_latent = torch.cat([obj_class, z_latent_obj],dim=-1)
    conditioned_obj_latent = torch.cat([nodes, conditioned_obj_latent],dim=-1)

    # part sampling
    z_latent_part = torch.normal(torch.zeros([batch_size,latent_dims]))
    conditioned_part_latent = torch.cat([conditioned_obj_latent, z_latent_part],dim=-1)


    x_bbx, x_lbl, _, _, x_bbx_refined = part_decoder(
        conditioned_part_latent, E, refine_iter=refine_iter)

    #  vae(adj_true, node_data_transformed, X_obj_true_transformed, label_true , class_true, variational, coupling, refine_iter=2)
    
    return x_bbx_refined, X_obj
    

In [14]:
[batch_val_loader] = data_loading.load_data(obj_data_postfix = '_mask_data_obj_boundary_sqr',
                                                              part_data_post_fix = '_mask_data_scaled_sqr',
                                                              file_postfix = '_combined_mask_data',
                                                              seed=345,
                                                              batch_size=batch_size,
                                                              validation=False
                                                              )

/opt/anaconda3/lib/python3.12/site-packages/torch_geometric/deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


In [15]:
#testing loop
model_path = ('/Users/amrutamuthal/Documents/training_runs/model/'+run_prefix+'/Obj-Box-'
                        +str(learning_rate)
                        +'-batch-'+str(batch_size)
                        +'-h1-'+str(hidden1)
                        +'-h2-'+str(hidden2)
                        +'-h3-'+str(hidden3)+'-test')
summary_path = ('/Users/amrutamuthal/Documents/training_runs/run/'+run_prefix+'/Obj-Box-'
                        +str(learning_rate)
                        +'-batch-'+str(batch_size)
                        +'-h1-'+str(hidden1)
                        +'-h2-'+str(hidden2)
                        +'-h3-'+str(hidden3)+'-test')

write_tensorboard = False
if write_tensorboard:
    writer = SummaryWriter(summary_path)

vae = TwoStageAutoEncoder(latent_dims,
                          num_nodes,
                          bbx_size,
                          num_classes,
                          label_shape,
                          hidden1,
                          hidden2,
                          hidden3,
                          dense_hidden1,
                          dense_hidden2,
                          adaptive_margin,
                          output_log,
                          area_encoding,
                          coupling,
                          obj_bbx_conditioning,
                          use_fft_on_bbx,
                          use_gcn_in_decoder,
                        )

vae.load_state_dict(torch.load(model_path+ '/model_weights.pth'))

part_decoder = vae.gcn_decoder
image_shape = [num_nodes, bbx_size]
global_step = 250000
class_dict = {0:'cow', 1:'sheep', 2:'bird', 3:'person', 4:'cat', 5:'dog', 6:'horse'}
res_dfs = []
actual_layouts = []
predicted_layouts = []

for i, val_data in enumerate(batch_val_loader, 0):
    
    val_data
    node_data_true = val_data.x
    label_true = node_data_true[:,:1]
    y_true = val_data.y

    class_true = y_true[:, :num_classes]
    X_obj_true = y_true[:, num_classes:]
    X_obj_true_transformed = X_obj_true[:,2:]-X_obj_true[:,:2]
    adj_true = val_data.edge_index
    class_true  = torch.flatten(class_true)
    curr_batch_size = node_data_true.shape[0]//num_nodes
    output = inference(X_obj_true_transformed, part_decoder, class_true, label_true, curr_batch_size, latent_dims,E=adj_true , refine_iter=2)
    node_data_pred_test = output[0]
    node_data_pred_test_new = torch.clip(torch.cat(
            [node_data_pred_test[:, :, :-2],
             node_data_pred_test[:, :, -2:] + node_data_pred_test[:, :, -4:-2]], axis=-1), min=0.0, max=1.0)
    X_obj_pred_test = output[1]

    actual_layouts.append(np.reshape(node_data_true.detach().cpu().numpy(),(curr_batch_size, num_nodes, 5)))
    
    
    node_true, node_pred, all_metrics = metrics.get_metrics(node_data_true, X_obj_true, node_data_pred_test_new,
                               X_obj_true,
                               label_true,
                               class_true,
                               num_nodes,
                               num_classes,
                               return_boxes = True)
    # actual_layouts.append(np.reshape(node_true, (curr_batch_size, num_nodes, 5)))
    predicted_layouts.append(node_pred)
    res_dfs.append(all_metrics)
    
    if write_tensorboard:
        
        for j in range(int(len(node_data_true)/num_nodes)):
            image = plot_utils.plot_bbx((node_data_true[num_nodes*j:num_nodes*(j+1)
                                                       ,1:5]*label_true[num_nodes*j:num_nodes*(j+1)]).detach().to("mps").numpy().astype(float))/255
            pred_image = plot_utils.plot_bbx((node_data_pred_test[j]*label_true[num_nodes*j:num_nodes*(j+1)]).detach().to("mps").numpy()).astype(float)/255

            writer.add_image('test_result/images/'+str(j)+'-input/', image, global_step, dataformats='HWC')  
            writer.add_image('test_result/images/'+str(j)+'-generated/', pred_image, global_step, dataformats='HWC')
            

result = pd.concat(res_dfs)
result['obj_class'] = np.where(result['obj_class'].isna(), 0, result['obj_class'])
result['obj_class'] = result['obj_class'].astype('int')
result['obj_class'].replace(class_dict, inplace=True)
result.where(result['part_labels']!=0, np.NaN, inplace=True)
result['part_labels'] = np.where(result['part_labels'].isna(), 0, result['part_labels'])
result['part_labels'] = result['part_labels'].astype('int')
result['id'] = np.repeat(np.array(list(range(int(len(result)/num_nodes)))), 16)

if write_tensorboard:
    writer.flush()
    writer.close()

bird_labels = {'head':1 , 'torso':2, 'neck':3, 'lwing':4, 'rwing':5, 'lleg':6, 'lfoot':7, 'rleg':8, 'rfoot':9, 'tail':10}
cat_labels = {'head':1, 'torso':2, 'neck':3, 'lfleg':4, 'lfpa':5, 'rfleg':6, 'rfpa':7, 'lbleg':8, 'lbpa':9, 'rbleg':10, 'rbpa':11, 'tail':12}
cow_labels = {'head':1,'lhorn':2, 'rhorn':3, 'torso':4, 'neck':5, 'lfuleg':6, 'lflleg':7, 'rfuleg':8, 'rflleg':9, 'lbuleg':10, 'lblleg':11, 'rbuleg':12, 'rblleg':13, 'tail':14}
dog_labels = {'head':1,'torso':2, 'neck':3, 'lfleg':4, 'lfpa':5, 'rfleg':6, 'rfpa':7, 'lbleg':8, 'lbpa':9, 'rbleg':10, 'rbpa':11, 'tail':12, 'muzzle':13}
horse_labels = {'head':1,'lfho':2, 'rfho':3, 'torso':4, 'neck':5, 'lfuleg':6, 'lflleg':7, 'rfuleg':8, 'rflleg':9, 'lbuleg':10, 'lblleg':11, 'rbuleg':12, 'rblleg':13, 'tail':14, 'lbho':15, 'rbho':16}
person_labels = {'head':1, 'torso':2, 'neck': 3, 'llarm': 4, 'luarm': 5, 'lhand': 6, 'rlarm':7, 'ruarm':8, 'rhand': 9, 'llleg': 10, 'luleg':11, 'lfoot':12, 'rlleg':13, 'ruleg':14, 'rfoot':15}
sheep_labels = cow_labels
part_labels_combined_parts = {'bird': bird_labels, 'cat': cat_labels, 'cow': cow_labels, 'dog': dog_labels, 'sheep': sheep_labels, 'horse':horse_labels,'person':person_labels}

for k, v in part_labels_combined_parts.items():
    new_map = {}
    for pk, pv in v.items():
        new_map[pv]=pk
    part_labels_combined_parts[k] = new_map
    
for k, v in part_labels_combined_parts.items():
    result.loc[result['obj_class']==k, ['part_labels']] = result.loc[result['obj_class']==k,['part_labels']].replace(v).copy()

result.to_csv(model_path+'/raw_test_metrics.csv')
# res_obj_level = result.groupby(['obj_class', 'id']).mean().reset_index()
# res_obj_level.groupby(['obj_class']).mean().reset_index()[['obj_class', 'IOU', 'MSE']].to_csv(model_path+'/obj_level_metrics.csv')
# result.groupby(['obj_class', 'part_labels']).mean().reset_index()[['obj_class', 'part_labels',  'IOU', 'MSE']].to_csv(model_path+'/part_level_metrics.csv')

torch.Size([128, 2]) torch.Size([128, 7])
torch.Size([128, 2]) torch.Size([128, 7])
torch.Size([128, 2]) torch.Size([128, 7])
torch.Size([128, 2]) torch.Size([128, 7])
torch.Size([128, 2]) torch.Size([128, 7])
torch.Size([52, 2]) torch.Size([52, 7])


/var/folders/ww/3nytfk1917b8nz7dchtckxnm0000gn/T/ipykernel_22666/1327200480.py:38: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  vae.load_state_dict(torch.load(model_path+ '

In [17]:
test_pred = np.concatenate(predicted_layouts)
test_true = np.concatenate(actual_layouts)
test_true.shape

labels = np.where(test_true[:, :, :1]>0, 1, 0)
test_pred = np.concatenate([labels, labels * test_pred], axis=-1)
test_pred.shape
# label_true

(692, 16, 5)

In [18]:

np.save(model_path+'/layout_preds.npy' ,test_pred ,allow_pickle=True)


In [ ]:
model_path = ('D:/meronym_data/model/'+run_prefix+'/Obj-Box-'
                        +str(learning_rate)
                        +'-batch-'+str(batch_size)
                        +'-h1-'+str(hidden1)
                        +'-h2-'+str(hidden2)
                        +'-h3-'+str(hidden3)+'-test')
summary_path = ('D:/meronym_data/runs/'+run_prefix+'/Obj-Box-'
                        +str(learning_rate)
                        +'-batch-'+str(batch_size)
                        +'-h1-'+str(hidden1)
                        +'-h2-'+str(hidden2)
                        +'-h3-'+str(hidden3)+'-test')

result = pd.read_csv(model_path+'/raw_metrics.csv')
result.groupby(['obj_class', 'part_labels']).mean().reset_index()[['obj_class', 'part_labels',
                                                                   'IOU', 'MSE']]

In [77]:
res_obj_level = result.groupby(['obj_class', 'id']).mean().reset_index()
res_obj_level.groupby(['obj_class']).mean().reset_index()[['obj_class', 'IOU', 'MSE']]

TypeError: agg function failed [how->mean,dtype->object]

In [56]:
res_obj_level[['IOU', 'MSE']].mean()

NameError: name 'res_obj_level' is not defined

In [ ]:
X_obj_pred[:10,2:]-X_obj_pred[:10,:2]

In [ ]:
X_obj_true[:10, 2:]-X_obj_true[:10, :2]

In [ ]:
res_obj_level.to_csv(model_path+'/obj_metrics.csv')

In [19]:
result.mean().reset_index()

TypeError: can only concatenate str (not "int") to str

In [ ]:
outfile = 'D:/meronym_data/generate_boxes.npy'
with open(outfile, 'wb') as pickle_file:
    pred_boxes = np.concatenate(pred_boxes)
    pickle.dump(pred_boxes, pickle_file)
outfile = 'D:/meronym_data/generate_boxesobj_class.npy'
with open(outfile, 'wb') as pickle_file:
    pickle.dump(classes,pickle_file)

In [53]:
temp_x = np.load("/Users/amrutamuthal/Documents/training_data/layout_gen/X_test_combined_mask_data.np", allow_pickle=True)
temp_adj = np.load("/Users/amrutamuthal/Documents/training_data/layout_gen/adj_test_combined_mask_data.np", allow_pickle=True)

In [54]:
temp_x.shape

(692, 16, 5)

In [68]:
def get_obj_boundary
temp_x[0][:, 1]


array([0.50454545, 0.51818182, 0.51969697, 0.26363636, 0.67424242,
       0.62575758, 0.64090909, 0.66212121, 0.67272727, 0.65      ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        ])

In [59]:
for data in batch_val_loader:
    print(len(data))

128
128
128
128
128
52
